# Check Initial Conditions Formula

An auxiliary file used to develop and test the function which is used for the generation of the initial conditions

In [1]:
from __future__ import annotations

import math
from dataclasses import dataclass, field
import numpy as np 

@dataclass
class PendulumParams:
    G: float = 9.8
    L1: float = 2.0
    L2: float = 2.0
    M1: float = 1.0
    M2: float = 1.0
    t_stop: float = 1000.0
    dt: float = 0.01

    @property
    def total_length(self) -> float:
        return self.L1 + self.L2

def generate_initial_conditions(
    E: float,
    n: int,
    params: PendulumParams
) -> list[tuple[float, float, float, float]]:
    """Function to generate n sets of initial conditions (th1, w1, th2, w2) all satisfying the same initial energy E"""
    inits: list[tuple[float, float, float, float]] = []

    # 1. Input parameters
    m1, m2 = params.M1, params.M2
    l1, l2 = params.L1, params.L2
    g = params.G
    M = m1 + m2

    while len(inits) < n:
        # 2. Random Angles
        th1 = np.random.uniform(-np.pi, np.pi)
        th2 = np.random.uniform(-np.pi, np.pi)

        # 3. Check Potential Energy Viability
        V = -M * g * l1 * np.cos(th1) - m2 * g * l2 * np.cos(th2)
        
        # If Potential Energy is greater than Total Energy, kinetic energy would be negative (impossible)
        if V > E:
            continue

        # 4. Determine Kinetic Energy Bounds (w1_max)
        # Using the discriminant condition B^2 - AC >= 0
        delta = th1 - th2
        term_denominator = l1**2 * (m1 + m2 * (np.sin(delta)**2))
        
        # Ensure we don't divide by zero (though physically unlikely with mass > 0)
        if term_denominator <= 0:
            continue
            
        w1_max = np.sqrt((2 * (E - V)) / term_denominator)

        # 5. Sample w1
        w1 = np.random.uniform(-w1_max, w1_max)

        # 6. Calculate Coefficients (A, B, C)
        # From Eq 11: A*w2^2 + 2*B*w2 + C = 0
        A = m2 * (l2**2)
        B = m2 * l1 * l2 * w1 * np.cos(delta)
        C = M * (l1**2) * (w1**2) - 2 * (E - V)

        # 7. Solve for w2
        discriminant = B**2 - A * C
        
        # Numerical stability check: strictly, discriminant >= 0 due to w1_max calculation, 
        # but floating point errors might produce slightly negative numbers near zero.
        if discriminant < 0:
            discriminant = 0.0
            
        root = np.sqrt(discriminant)
        
        # Randomly select (+) or (-) solution
        sign = np.random.choice([-1.0, 1.0])
        w2 = (-B + sign * root) / A

        # 8. Store Valid State
        # Order: [theta_1, w1, theta_2, w2]
        inits.append((th1, w1, th2, w2))

    return inits

In [2]:
def verify_energy(th1, w1, th2, w2, params: PendulumParams):
    """Function that checks the energy of a pendulum given its state"""
    m1, m2 = params.M1, params.M2
    l1, l2 = params.L1, params.L2
    g = params.G
    M = m1 + m2

    # Potential Energy
    V = -M * g * l1 * np.cos(th1) - m2 * g * l2 * np.cos(th2)

    # Kinetic Energy
    # Derived from Eq (8): 
    # T = 0.5*M*l1^2*w1^2 + 0.5*m2*l2^2*w2^2 + m2*l1*l2*w1*w2*cos(th1-th2)
    T = (0.5 * M * (l1**2) * (w1**2) + 
         0.5 * m2 * (l2**2) * (w2**2) + 
         m2 * l1 * l2 * w1 * w2 * np.cos(th1 - th2))

    E = V + T
    return E

def potential_energy(th1, w1, th2, w2, params: PendulumParams):
    """Function that checks the energy of a pendulum given its state"""
    m1, m2 = params.M1, params.M2
    l1, l2 = params.L1, params.L2
    g = params.G
    M = m1 + m2

    # Potential Energy
    V = -M * g * l1 * np.cos(th1) - m2 * g * l2 * np.cos(th2)

    return V

In [10]:
init_conditions = generate_initial_conditions(120, 10, PendulumParams)
# print(init_conditions)

for IC in init_conditions:
    print(f"Initial Conditions: th1={IC[0]:6.4f}, w1={IC[1]:6.4f}, th2={IC[2]:6.4f}, w2={IC[3]:6.4f}, PE: {potential_energy(*IC, PendulumParams):6.4f}, E = {verify_energy(*IC, PendulumParams)}")

Initial Conditions: th1=-0.5411, w1=-3.9607, th2=2.0589, w2=-10.6292, PE: -24.4077, E = 120.00000000000003
Initial Conditions: th1=-0.2681, w1=2.0263, th2=-2.3409, w2=-7.0758, PE: -24.1542, E = 120.00000000000006
Initial Conditions: th1=-0.7153, w1=2.3758, th2=-1.8264, w2=-8.9371, PE: -24.6353, E = 120.0
Initial Conditions: th1=2.1140, w1=-4.3476, th2=2.1807, w2=-0.6890, PE: 31.4889, E = 120.0
Initial Conditions: th1=-0.7139, w1=-0.6549, th2=-2.7365, w2=-8.3501, PE: -11.6137, E = 120.00000000000001
Initial Conditions: th1=2.3806, w1=0.9880, th2=0.1059, w2=7.9885, PE: 8.8967, E = 120.0
Initial Conditions: th1=1.8523, w1=-4.0055, th2=-2.6235, w2=2.9127, PE: 27.9188, E = 120.0
Initial Conditions: th1=2.2411, w1=-1.5276, th2=-1.8193, w2=-7.3771, PE: 29.1729, E = 120.0
Initial Conditions: th1=-1.3113, w1=0.9327, th2=-0.6807, w2=-9.2074, PE: -25.2889, E = 119.99999999999997
Initial Conditions: th1=-0.8752, w1=-6.9392, th2=-0.6463, w2=1.3037, PE: -40.7668, E = 119.99999999999997
